<a href="https://colab.research.google.com/github/Dorilitre/RecursosParaJogos/blob/master/Bot_Total_Corner_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# VÍDEOS DE CRIAÇÃO DO CÓDIGO
'''
PARTE 1 - https://youtu.be/XoeRR6YIGN4
PARTE 2 - https://youtu.be/2zYYCo7_Scs

'''
import requests
import pandas as pd
import numpy as np
import re
from time import sleep

In [ ]:
# Oxylabs
# LINK DO VÍDEO PARA CRIAR USUARIO E SENHA OXYLABS -> https://youtu.be/mO6YFoflMh0

url = 'https://realtime.oxylabs.io/v1/queries'
auth = ('', '')
headers = {'Content-Type': 'application/json'}
data = {
    'source': 'universal',
    'url': 'https://www.totalcorner.com/match/today',
    'geo-localização': 'Brazil',
    'renderizar': 'html'
}

response = requests.post(url, auth=auth, headers=headers, json=data)
html = response.json()['results'][0]['content']


In [ ]:
df = pd.read_html(html)[0]

In [ ]:
df.dropna(axis=1,how='all',inplace=True)

In [ ]:
df.dropna(axis=0,how='any',inplace=True)

In [ ]:
df.drop(columns=['Handicap','Goal Line','Analysis','Time'],inplace=True)

In [ ]:
df.rename(columns={'Unnamed: 3':'Cronometro'},inplace=True)

In [ ]:
df['Cronometro'] = df['Cronometro'].apply(lambda a: 45 if a == 'Half' else a).astype('int64')

In [ ]:
df['Corner'] = df['Corner'].apply(lambda a: re.findall(r'\d{1,} - \d{1,}',a))

In [ ]:
df['Corner'] = df['Corner'].apply(lambda a: list(map(int,a[0].split(' - '))))

In [ ]:
df['Dangerous Attack'] = df['Dangerous Attack'].apply(lambda a: re.findall(r'\d{1,} - \d{1,}',a))

In [ ]:
df['Shots'] = df['Shots'].apply(lambda a: re.findall(r'\d{1,} - \d{1,}',a))

In [ ]:
df['Dangerous Attack'] = df['Dangerous Attack'].apply(lambda a: a if a != [] else np.nan)
df['Corner'] = df['Corner'].apply(lambda a: a if a != [] else np.nan)
df['Shots'] = df['Shots'].apply(lambda a: a if a != [] else np.nan)

In [ ]:
df.dropna(axis=0, how = 'any',inplace=True)

In [ ]:
df['Shots'] = df['Shots'].apply(lambda a: list(map(int,a[0].split(' - '))))
df['Dangerous Attack'] = df['Dangerous Attack'].apply(lambda a: list(map(int,a[0].split(' - '))))

In [ ]:
df.head()

,League,Cronometro,Home,Score,Away,Corner,Dangerous Attack,Shots
2,Malaysia President Cup,73,2 Johor Darul Ta'zim III U21,2 - 0,Perak U21 1,"[5, 2]","[32, 17]","[10, 3]"
3,Brunei Super League,74,AKSE Bersatu,5 - 1,BSRC 1,"[10, 5]","[54, 37]","[12, 4]"
4,Malaysia President Cup,75,1 Sabah U21,3 - 1,Kelantan U21 1,"[9, 1]","[31, 22]","[14, 4]"
5,Europe Friendlies,45,SCR Altach,0 - 0,FC 08 Homburg,"[6, 1]","[35, 13]","[7, 1]"
6,Brazil U20 League,45,1 CA Pernambucano U20,0 - 1,Sport Recife U20 1,"[3, 1]","[37, 38]","[5, 1]"


In [ ]:
# CRIE SEU BOT TELEGRAM
# lINK -> https://youtu.be/6xfUG8bghCw
def enviar_mensagem(msg):
  bot_token = ''
  chat_id = ''
  url = f'https://api.telegram.org/bot{bot_token}/sendMessage?chat_id={chat_id}&text={msg}'
  requests.post(url)

In [ ]:
mensagens_enviadas = []

In [ ]:

for i in range(len(df)):
  j = df.iloc[i]
  tempo = j['Cronometro']

  appm_home = round(j['Dangerous Attack'][0]/tempo,1)
  appm_away = round(j['Dangerous Attack'][1]/tempo,1)

  cg_home = j['Corner'][0] + j['Shots'][0]
  cg_away = j['Corner'][1] + j['Shots'][1]

  condicao_home = appm_home >= 0.5 and cg_home >= 10
  condicao_away = appm_away >= 0.5 and cg_away >= 10

  t1 = 0 < tempo < 45
  t2 = 45 < tempo < 90


  if (condicao_home or condicao_away) and (t1 or t2):
    if t1:
      t = 't1'
    else:
      t= 't2'

    home = re.sub(r'^\d{1} ','',j['Home'])
    away = re.sub(r'\d{1}$','',j['Away'])
    if f'{home} x {away}_{t}' not in mensagens_enviadas:
      msg = (f'''

  Liga:{j["League"]}
  jogo: {home} x {away}
  Placar: {j["Score"]}
  Appm: {appm_home} - {appm_away}
  CG: {cg_home} - {cg_away}
  ''')
      enviar_mensagem(msg)
      mensagens_enviadas.append(f'{home} x {away}_{t}')
      sleep(2)





In [ ]:
mensagens_enviadas

['AKSE Bersatu x BSRC _t2']